In [1]:
from Preprocessing import Preprocessing
from utils import Behaviour, Baseline
import Experiments
import matplotlib
matplotlib.use('Agg') # Prevent drawings in this notebook
import matplotlib.pyplot as plt
from multiprocessing import Pool
import math

In [2]:
bids_root = "../data/"
preprocessing = Preprocessing(bids_root, "cache")
preprocessing.prepare_channels()

In [ ]:
combinations = [(i, behaviour) for i in range(65) for behaviour in Behaviour]
combinations.sort(key=lambda x: x[0])

process_count = 16
inputs = []
step_size = math.ceil(len(combinations)/process_count)
for i in range(process_count):
    inputs.append(combinations[step_size*i:step_size*(i+1)])

with Pool(process_count) as p:
    result = p.map(Experiments.run, inputs)
# Experiments.run(combinations)

  0%|          | 0/57 [00:00<?, ?it/s]

In [ ]:
# join results
peak_to_troughs = {}
for behavior in Behaviour:
    peak_to_troughs[behavior] = {}
for i in range(len(result)):
    for behavior in result[i]:
        peak_to_troughs[behavior].update(result[i][behavior])  # reverse to keep it ordered

In [ ]:
def plot_peak_to_trough_differences(behaviour, baseline, mode):
    data = []
    for electrode_index, peak_to_trough_value in peak_to_troughs[behaviour].items():
        data.append((electrode_index, peak_to_trough_value[mode][baseline]))
    title = f"Lower Peak-to-Trough Differences for {behaviour.value} ({'Raw' if mode == 0 else 'ICA'}, {baseline.value})"
    sorted_values = sorted(data, key=lambda x:x[1])[:20]
    labels, values = zip(*sorted_values)
    plt.figure()
    plt.barh(range(len(values)), values, color='skyblue')
    plt.xlabel("Peak-to-Trough Difference")
    plt.ylabel("Electrode")
    plt.yticks(range(len(labels)), labels)
    plt.title(title)
    plt.savefig(f"../Results/{title}.png")
    plt.close()    

In [ ]:
for behavior in peak_to_troughs:
    for baseline in Baseline:
        plot_peak_to_trough_differences(behavior, baseline, 0)
        plot_peak_to_trough_differences(behavior, baseline, 1)